In [139]:
import re
import os
import pandas as pd
import webbrowser
from pathlib import Path

In [141]:
# Exported player dataset from FM24
html_file = "path/roster.html"

In [143]:
# Parse HTML and extract tables
rlist = pd.read_html(html_file, header=0, encoding="utf-8", keep_default_na=False)
roster_data = rlist[0]

In [145]:
# Define the role structure(New rolls can be added or removed from this section along with their weights)
roles = {
    'Sweeper Keeper Defend': {
        'key': ['Agi', 'Ref'],
        'green': ['Cmd', 'Kic', '1v1', 'Ant', 'Cnt', 'Pos'],
        'blue': ['Aer', 'Fir', 'Han', 'Pas', 'TRO', 'Dec', 'Vis', 'Acc']
    },
    'Central Defender Defend': {
        'key': ['Pac', 'Jum', 'Cmp', 'Cnt', 'Bra', 'Agg', 'Ant'],
        'green': ['Hea', 'Mar', 'Tck', 'Pos', 'Str'],
        'blue': ['Acc', 'Dec']
    },
    'Ball Playing Defender Defend': {
        'key': ['Acc', 'Pac', 'Jum', 'Cmp'],
        'green': ['Hea', 'Mar', 'Pas', 'Tck', 'Pos', 'Str'],
        'blue': ['Fir', 'Tec', 'Agg', 'Ant', 'Bra', 'Cnt', 'Dec', 'Vis']
    },
    'Full Back Support': {
        'key': ['Acc', 'Pac', 'Sta', 'Wor', 'Cro', 'Dri', 'Pas', 'Agi', 'Tec'],
        'green': ['Mar', 'Tck', 'Ant', 'Cnt', 'Pos', 'Tea', 'Acc'],
        'blue': ['Dec']
    },
    'Inverted Wing Back Support': {
        'key': ['Sta', 'Wor', 'Mar', 'Tec'],
        'green': ['Fir', 'Pas', 'Tck', 'Cmp', 'Dec', 'Tea'],
        'blue': ['Pac', 'Ant', 'Cnt', 'OtB', 'Pos', 'Vis', 'Agi']
    },
    'Wing Back Support': {
        'key': ['Acc', 'Pac', 'Sta', 'Wor', 'Cro', 'Dri', 'Pas', 'Agi', 'Tec'],
        'green': ['Cro', 'Dri', 'Mar', 'Tck', 'OtB', 'Tea', 'Acc'],
        'blue': ['Fir', 'Pas', 'Tec', 'Ant', 'Cnt', 'Dec', 'Pos', 'Bal']
    },
    'Central Midfielder Support': {
        'key': ['Acc', 'Pac', 'Sta', 'Wor', 'Tec', 'Vis', 'OtB'],
        'green': ['Fir', 'Pas', 'Tck', 'Dec', 'Tea'],
        'blue': ['Ant', 'Cmp', 'Cnt']
    },
    'Ball Winning Midfielder Support': {
        'key': ['Wor', 'Sta', 'Acc', 'Pac'],
        'green': ['Tck', 'Agg', 'Ant', 'Tea'],
        'blue': ['Mar', 'Pas', 'Bra', 'Cnt', 'Agi', 'Str']
    },
    'Advanced Playmaker Support': {
        'key': ['Acc', 'Pac', 'Sta', 'Wor'],
        'green': ['Fir', 'Pas', 'Tec', 'Cmp', 'Dec', 'OtB', 'Tea', 'Vis'],
        'blue': ['Dri', 'Ant', 'Fla', 'Agi']
    },
    'Deep Lying Playmaker Support': {
        'key': ['Wor', 'Sta', 'Fir', 'Pas', 'Tec', 'Cmp', 'Dec', 'Tea', 'Vis', 'Bal', 'OtB', 'Pos', 'Ant'],
        'green': ['Fir', 'Pas', 'Tec', 'Cmp', 'Dec', 'Tea', 'Vis'],
        'blue': ['Pac', 'Acc']
    },
    'Winger Support': {
        'key': ['Acc', 'Pac', 'Sta', 'Wor', 'Fir', 'Pas'],
        'green': ['Cro', 'Dri', 'Tec', 'Agi'],
        'blue': ['OtB', 'Bal']
    },
    'Inside Forward Attack': {
        'key': ['Acc', 'Pac', 'Sta', 'Wor'],
        'green': ['Dri', 'Fin', 'Fir', 'Tec', 'Ant', 'OtB', 'Agi'],
        'blue': ['Lon', 'Pas', 'Cmp', 'Fla', 'Bal']
    },
    'Inverted Winger Attack': {
        'key': ['Acc', 'Sta', 'Wor'],
        'green': ['Cro', 'Dri', 'Pas', 'Tec', 'Agi'],
        'blue': ['Fir', 'Lon', 'Ant', 'Cmp', 'Dec', 'Fla', 'OtB', 'Vis', 'Bal', 'Pac']
    },
    'Advanced Forward Attack': {
        'key': ['Acc', 'Pac', 'Fin'],
        'green': ['Dri', 'Fir', 'Tec', 'Cmp', 'OtB'],
        'blue': ['Pas', 'Ant', 'Dec', 'Wor', 'Agi', 'Bal', 'Sta']
    }
}

In [159]:
def calculate_role_ratings(df, roles_dict):
    """
    Calculates role ratings for each player in the DataFrame based on attribute weights.

    Parameters:
    - df: pandas DataFrame containing player attributes
    - roles_dict: dictionary with roles as keys and 'key', 'green', 'blue' attributes as lists

    Returns:
    - df: new DataFrame with role rating columns added
    """
    # Define weights
    KEY_WEIGHT, GREEN_WEIGHT, BLUE_WEIGHT = 5, 3, 1

    df = df.copy()

    for role, attrs in roles_dict.items():
        for cat1, cat2 in [('key', 'green'), ('key', 'blue'), ('green', 'blue')]:
            overlap = set(attrs[cat1]) & set(attrs[cat2])
            if overlap:
                raise ValueError(f"Attribute(s) {overlap} found in both '{cat1}' and '{cat2}' for role {role}")

        missing = [col for group in attrs.values() for col in group if col not in df.columns]
        if missing:
            raise ValueError(f"Missing attributes for role {role}: {missing}")

        key = df[attrs['key']].sum(axis=1) * KEY_WEIGHT
        green = df[attrs['green']].sum(axis=1) * GREEN_WEIGHT
        blue = df[attrs['blue']].sum(axis=1) * BLUE_WEIGHT

        total_weight = (
            len(attrs['key']) * KEY_WEIGHT +
            len(attrs['green']) * GREEN_WEIGHT +
            len(attrs['blue']) * BLUE_WEIGHT
        )

        df[role] = ((key + green + blue) / total_weight).round(1)

    return df


In [155]:
# Generate versioned CSV filename from the input HTML path
def generate_versioned_csv_filename(input_path):

    base = os.path.basename(input_path)
    name, ext = os.path.splitext(base)

    match = re.search(r"^(.*)_v(\d+)$", name)
    if match:
        root_name = match.group(1)
        version = int(match.group(2)) + 1
    else:
        root_name = name
        version = 1

    new_name = f"{root_name}_v{version}.csv"
    return os.path.join(os.path.dirname(input_path), new_name)

In [ ]:
# Generate and save the updated CSV file
csv_output_file = generate_versioned_csv_filename(html_file.replace('.html', '.csv'))
roster_data.to_csv(csv_output_file, index=False, encoding="utf-8-sig")
print("CSV saved as:", csv_output_file)